In [1]:
import netCDF4
import operator
import xarray as xr
import dask
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as patches
from matplotlib import ticker
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import scipy
from scipy import signal
import math
from mpl_toolkits.basemap import Basemap
bm = Basemap() 

#added from Lutsko Github
import spectrum 

/export/home/gmooers/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-7109723e9a77>", line 21, in <module>
    import spectrum
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/spectrum/__init__.py", line 19, in <module>
    from .datasets import *
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/spectrum/datasets.py", line 148, in <module>
    dolphin_filename = spectrum_data("DOLPHINS.wav")
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/spectrum/datasets.py", line 136, in spectrum_data
    import pkg_resources
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/pkg_resources/__init__.py", line 84, in <module>
    __import__('pkg_resources.extern.packaging.requirements')
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/pkg_res

KeyboardInterrupt: 

In [ ]:
#Good_July.nc
path_to_file = '/fast/gmooers/Models/CAM_July.nc'
real_ds = xr.open_dataset(path_to_file)


print('files imported')

In [ ]:
#heat_real_ds = real_ds.PTEQ[:, :].values
moist_real_ds = real_ds.PTEQ.values
heat_real_ds = real_ds.PTTEND.values

times = real_ds.time.values

lats = real_ds.lat.values

lons = real_ds.lon.values

In [ ]:
x = 144
y = 96
z = 30
t = int(len(heat_real_ds)/(x*y))

In [ ]:
spacing = 1/96

In [ ]:
others = netCDF4.Dataset("/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-72000.nc")
plev = np.array(others.variables['lev'])
ps = np.array(others.variables['PS'])
g = 9.81 #m/s^2
L = 2256000.0
cp = 1004.0
#print(plev)
hyai = np.array(others.variables['hyai'])
hybi = np.array(others.variables['hybi'])
PS = 1e5
P0 = 1e5
P = P0*hyai+PS*hybi # Total pressure [Pa]
dp = P[1:]-P[:-1] # Differential pressure [Pa]
#convert from k/s to w/m^2
heat_pressure_weighted_targets = heat_real_ds*dp[None, :,None, None]*cp/g
moist_pressure_weighted_targets = moist_real_ds*dp[None, :,None, None]*L/g
print('made it')

In [ ]:
def spectrum_gen(h, dt):
    nt = len(h)
    #Get half the length of the time series to avoid redudant information
    npositive = nt//2
    pslice = slice(1, npositive)
    #Get frequencies
    freqs = np.fft.fftfreq(nt, d=dt)[pslice] 
    #perform the fft 
    ft = np.fft.fft(h)[pslice]
    #remove imaginary componant of the fft and square
    psraw = np.conjugate(ft) *ft
    #double to account for the negative half that was removed above
    psraw *= 2.0
    #Normalization for the power spectrum
    psraw /= nt**2
    #Go fro mthe Power Spectrum to Power Density
    psdraw = psraw * dt * nt
    return freqs, psraw, psdraw
    
#spectrum_gen(s1, t[1]-t[0])

In [ ]:
def spectrum_generator(targets, levels, latitude_count, longitude_count, time_space):
    targ_freqs, targ_psraw, targ_psdraw = spectrum_gen(np.squeeze(targets[:,0,0,0]), time_space)
    depth = len(targ_psdraw)
    target_collector = np.zeros(shape=(depth, levels, latitude_count, longitude_count))
    target_collector[:,:,:] = np.nan
    counter = 0
    for i in range(levels):
        for j in range(latitude_count):
            for k in range(longitude_count):
                target = np.squeeze(targets[:,i,j,k])
                targ_freqs, targ_psraw, targ_psdraw = spectrum_gen(target, time_space)
                target_collector[:,i,j,k] = targ_psdraw
        
    return targ_freqs, target_collector

heat_targ_freqs,  heat_target_collector = spectrum_generator(heat_pressure_weighted_targets, z, y, x, spacing)
moist_targ_freqs,  moist_target_collector = spectrum_generator(moist_pressure_weighted_targets, z, y, x, spacing)

In [ ]:
def plotter(target_array, frequency, title_name):
    plt.plot(1/frequency, target_array, 'r', alpha = 0.5, label = "CAM5")
    plt.legend()
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('Period (Days)')
    h = plt.ylabel(r'$\frac{w^2}{m^4*day}$', fontsize = 15)
    h.set_rotation(0)
    plt.title("Power Spectrum Density for "+title_name)

In [ ]:
lat_list = lats
lon_list = lons

#method 1 - Hovmueller Github 
a = 6.37e6 #radius of the earth
dlat = np.abs(lat_list[1]-lat_list[0])
dlon = np.abs(lon_list[0]-lon_list[1])
gridCellWidth = (dlon*np.pi/180.)*a*np.cos(lat_list*np.pi/180.)

heat_lat_weighted_targs = heat_target_collector*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
heat_lat_weighted_targs = np.nanmean(heat_lat_weighted_targs, axis = 1)

moist_lat_weighted_targs = moist_target_collector*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
moist_lat_weighted_targs = np.nanmean(moist_lat_weighted_targs, axis = 1)

heat_pressure_weighted_target_output = np.nanmean(heat_lat_weighted_targs , axis = 1)
moist_pressure_weighted_target_output = np.nanmean(moist_lat_weighted_targs, axis = 1)

heat_target_avg = np.nanmean(heat_pressure_weighted_target_output, axis = 1)
moist_target_avg = np.nanmean(moist_pressure_weighted_target_output, axis = 1)

In [ ]:
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Updated_Weighted_CAM_Spectra_Moisture.npy",moist_target_avg)
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Updated_Weighted_CAM_Spectra_Heat.npy",heat_target_avg)
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Updated_CAM_Spectra_freqs_Moisture.npy", moist_targ_freqs)
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Updated_CAM_Spectra_freqs_Heat.npy", heat_targ_freqs)

In [ ]:

title = 'All locations'
plotter(heat_target_avg, heat_targ_freqs, title)
plotter(moist_target_avg, moist_targ_freqs, title)

# Apply land and ocean  masks

In [ ]:
path_to_file = '/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h0.2018-07.nc'
land_frac = xr.open_dataset(path_to_file)
land_f = np.squeeze(land_frac.LANDFRAC.values)

new_land_frac = np.zeros(shape=(144, 96))
new_land_frac[:,:] = np.nan

In [ ]:
print(heat_target_collector.shape)
print(land_f.shape)

In [ ]:
ocean_heat = np.zeros(shape=(len(heat_target_collector),len(heat_target_collector[0]),len(heat_target_collector[0][0]),len(heat_target_collector[0][0][0])))
ocean_heat[:,:,:,:] = np.nan
ocean_moist = np.zeros(shape=(len(heat_target_collector),len(heat_target_collector[0]),len(heat_target_collector[0][0]),len(heat_target_collector[0][0][0])))
ocean_moist[:,:,:,:] = np.nan
land_heat = np.zeros(shape=(len(heat_target_collector),len(heat_target_collector[0]),len(heat_target_collector[0][0]),len(heat_target_collector[0][0][0])))
land_heat[:,:,:,:] = np.nan
land_moist = np.zeros(shape=(len(heat_target_collector),len(heat_target_collector[0]),len(heat_target_collector[0][0]),len(heat_target_collector[0][0][0])))
land_moist[:,:,:,:] = np.nan

In [ ]:
print(np.count_nonzero(~np.isnan(heat_target_collector)))
print(np.count_nonzero(~np.isnan(moist_target_collector)))
print(np.count_nonzero(~np.isnan(land_heat)))
print(np.count_nonzero(~np.isnan(land_moist)))
print(np.count_nonzero(~np.isnan(ocean_heat)))
print(np.count_nonzero(~np.isnan(ocean_moist)))

In [ ]:
for i in range(len(heat_target_collector[0][0])):
    for j in range(len(heat_target_collector[0][0][0])):
        if land_f[i,j] >= 0.5:
            land_moist[:,:,i,j] = moist_target_collector[:,:,i,j]
            land_heat[:,:,i,j] = heat_target_collector[:,:,i,j]
        if land_f[i,j] < 0.5:
            ocean_moist[:,:,i,j] = moist_target_collector[:,:,i,j]
            ocean_heat[:,:,i,j] = heat_target_collector[:,:,i,j]

In [ ]:
print(np.count_nonzero(~np.isnan(heat_target_collector)))
print(np.count_nonzero(~np.isnan(moist_target_collector)))
print(np.count_nonzero(~np.isnan(land_heat)))
print(np.count_nonzero(~np.isnan(land_moist)))
print(np.count_nonzero(~np.isnan(ocean_heat)))
print(np.count_nonzero(~np.isnan(ocean_moist)))

In [ ]:
final_land_heat =land_heat*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
final_land_heat = np.nanmean(final_land_heat, axis = 2) 
final_land_heat = np.nanmean(final_land_heat, axis = 1) 
final_land_heat = np.nanmean(final_land_heat, axis = 1)

final_land_moist = land_moist*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
final_land_moist = np.nanmean(final_land_moist, axis = 2)
final_land_moist = np.nanmean(final_land_moist, axis = 1) 
final_land_moist = np.nanmean(final_land_moist, axis = 1)

final_ocean_heat = ocean_heat*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
final_ocean_heat = np.nanmean(final_ocean_heat, axis = 2)
final_ocean_heat = np.nanmean(final_ocean_heat, axis = 1) 
final_ocean_heat = np.nanmean(final_ocean_heat, axis = 1)

final_ocean_moist = ocean_moist*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
final_ocean_moist = np.nanmean(final_ocean_moist, axis = 2)
final_ocean_moist = np.nanmean(final_ocean_moist, axis = 1)
final_ocean_moist = np.nanmean(final_ocean_moist, axis = 1)

In [ ]:
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Weighted_CAM_Spectra_Moisture_Ocean.npy", final_ocean_moist)
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Weighted_CAM_Spectra_Moisture_Land.npy", final_land_moist)
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Weighted_CAM_Spectra_Heat_Land.npy", final_land_heat)
np.save("/fast/gmooers/Real_Geography_Manuscript/Compressed_Data/Weighted_CAM_Spectra_Heat_Ocean.npy", final_ocean_heat)

#np.save("/fast/gmooers/RG_Paper_Data/CAM_Spectra_freqs_Moisture.npy", targ_freqs)
#np.save("/fast/gmooers/RG_Paper_Data/Weighted_CAM_Spectra_Moisture_Land.npy",target_avg)
#np.save("/fast/gmooers/RG_Paper_Data/CAM_Spectra_freqs_Moisture_Land.npy", targ_freqs)

In [ ]:
plt.plot(final_land_heat)

In [ ]:
plt.plot(final_land_moist)

In [ ]:
plt.plot(final_ocean_heat)

In [ ]:
plt.plot(final_ocean_moist)